<a href="https://colab.research.google.com/github/SusannYY/Unemployment-Rate-Analysis/blob/main/Week4%20Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import json

### Choose the survey data IDs you want to retrieve from the API

1. Read through this [lists](https://www.bls.gov/data/#employment) of survey data \
2. Browse the Subjects(Inflation & Prices, Employment, ...), then browse the **Database Name** under certain Subject, and then choose your interested one \
3. To view the exact data or search more detailed categories under this data topic, you can click on **OneScreen** : ![image.png](https://www.bls.gov/images/icons/icon_large_one_screen.gif)
4. Take the **Labor Force Statistics (Current Population Survey - CPS)** under the **Employment** Subject as an example: \
By clicking on the green **OneScreen** button, you can be directed to this [link](https://data.bls.gov/PDQWeb/ln). \
5. You can then customize your own searching limits *(Eg. Sex: men; Race: White, Hispanic or Latino Ethnicity: All Origins; Age: 25 years and over, Education (25 years and over only): High School graduates, no college; Labor Force Status: Unemployment rate)* \
6. Then click on **Get Data** button, which will redirect you to some page like [this one](https://data.bls.gov/pdq/SurveyOutputServlet). You can see the **Series Id:**	LNU03000380 at the top description of this survey table. \
7. Save the ID and put into the `series_dict` dictionary below. \
8. You can repeate this searching process, select different combinations in step 5, or even find other useful data in the [main page](https://www.bls.gov/data/#employment). \
9. Record and input all the survey IDs you are going to use to the `series_dict` dictionary below. \
10. You can always check by searching survey ID [here](https://beta.bls.gov/dataQuery/search). \
10. This lab retrieve the three races' unemployment rate from 2013 to 2022 (Black, Hispanic, and White). **Remember to change certain parameters or variables based on the data you choose.** \
11. After running this following lab, you will eventually get a well-organized dataset containing all the surveys you want to look at and analyze on.

In [ ]:
series_dict = {'LNS14000006': 'Black', 'LNS14000009': 'Hispanic',
      'LNS14000003': 'White'}
series_list = list(series_dict.keys())

## Build API call

1. You can see that we can set `startyear` and `endyear` to any time as long as the whole interval is <= 20 years. \
2. This is the public API from US Bureau of Labor Statistics. \
3. You can have 25 API calls (pull_data process) within one day and 10 years' interval in each call, if you don't register. \
4. With registration, you can have 500 times of API calls and 20 years' interval in each call. \
5. And this is why we have `"registrationkey":"647837ae6003448b9488e8cb39c5b533"`. \
6. This is my personal `registrationkey` and you need to generate your own by registering [here](https://data.bls.gov/registrationEngine/). \
7. After that checkout your email and copy paste your unique key to replace mine `"647837ae6003448b9488e8cb39c5b533"` \
8. Check out this [frq](https://www.bls.gov/developers/api_faqs.htm#register2), [explanation](https://www.bls.gov/developers/api_signature_v2.htm#multiple), or [sample code](https://www.bls.gov/developers/api_python.htm) if further questions exist.

In [ ]:
def pull_data (series_id):
  # Specify json as content type to return
  headers = {'Content-type': 'application/json'}

  # Submit the list of series as data
  data = json.dumps({"seriesid": [series_id],"startyear":"2013", "endyear":"2022", "registrationkey":"647837ae6003448b9488e8cb39c5b533"})

  # Post request for the data
  p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
  # change p into json type for further querying
  json_data = json.loads(p.text)
  
  # query data
  results = json_data["Results"]["series"][0]["data"]
  return results

For `json_data["Results"]["series"][0]["data"]`, feel free to run the pull_data lines by lines and print out json_data to see how and why we query in this way

### Further query data

In [ ]:
df = [pd.DataFrame() for i in range(3)] # range(3) because we have three survey IDs selected
data = pd.DataFrame()                 # create an empty DataFrame
for j in range(3):
  results = pull_data(series_list[j]) # api call for each survey ID
  for i in np.arange(0,10*12-4, 1):   # might need to change based on years you choose
    year = results[i]["year"]         
    month = results[i]["period"][1:]  # this is to delete the first letter "M" in every month data
    month_name = results[i]["periodName"]
    date_list = year + "-" + month + "-01" 
    unemployment_rate = results[i]["value"]
    df[j] = df[j].append({series_dict.get(series_list[j])+"_unemployment": unemployment_rate},
                      ignore_index=True)
    if j == 0:
      data = data.append({"date_list": date_list},
                      ignore_index=True)

In [ ]:
results

In [ ]:
df[0]

,Black_unemployment
0,6.4
1,6.0
2,5.8
3,6.2
4,5.9
...,...
111,13.4
112,13.3
113,13.0
114,13.8


In [ ]:
data

,date_list
0,2022-08-01
1,2022-07-01
2,2022-06-01
3,2022-05-01
4,2022-04-01
...,...
111,2013-05-01
112,2013-04-01
113,2013-03-01
114,2013-02-01


In [ ]:
output = [data] + df
output

[      date_list
 0    2022-08-01
 1    2022-07-01
 2    2022-06-01
 3    2022-05-01
 4    2022-04-01
 ..          ...
 111  2013-05-01
 112  2013-04-01
 113  2013-03-01
 114  2013-02-01
 115  2013-01-01
 
 [116 rows x 1 columns],     Black_unemployment
 0                  6.4
 1                  6.0
 2                  5.8
 3                  6.2
 4                  5.9
 ..                 ...
 111               13.4
 112               13.3
 113               13.0
 114               13.8
 115               13.7
 
 [116 rows x 1 columns],     Hispanic_unemployment
 0                     4.5
 1                     3.9
 2                     4.3
 3                     4.3
 4                     4.1
 ..                    ...
 111                   9.0
 112                   9.0
 113                   9.3
 114                   9.7
 115                   9.7
 
 [116 rows x 1 columns],     White_unemployment
 0                  3.2
 1                  3.1
 2                  3.3
 3        

In [ ]:
# use concat to combine the arrays in "output"
final_data = pd.concat(output, ignore_index=False, axis=1) # axis=1 to concat by columns; ignore_index=False to preserve the head of each column
final_data

,date_list,Black_unemployment,Hispanic_unemployment,White_unemployment
0,2022-08-01,6.4,4.5,3.2
1,2022-07-01,6.0,3.9,3.1
2,2022-06-01,5.8,4.3,3.3
3,2022-05-01,6.2,4.3,3.2
4,2022-04-01,5.9,4.1,3.2
...,...,...,...,...
111,2013-05-01,13.4,9.0,6.7
112,2013-04-01,13.3,9.0,6.7
113,2013-03-01,13.0,9.3,6.7
114,2013-02-01,13.8,9.7,6.8


In [ ]:
final_data.to_csv('unemployment_rate_races.csv')